In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string, os
import nltk
import re
import keras
import random
import io
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adamax
import sys
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings("ignore")


<a id="2"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">LOADING DATA</p>
For this project, I have prepared a dataset of song lyrics. Let's load it and have a look.

<a id="3"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">DATA EXPLORATION</p>

**In this section, I will be:**
* Exploring the various artists in data
* Explore the number of songs and their corresponding information
* Explore the various words in lyrics via wordcloud 

So I have a total of 745 songs

**I will do a little feature engineering to extract more information on the songs such as:**
* Number of characters
* Number of words
* Number of lines

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# Detecting language took a lot of time. I am saving it as new csv file for future 
# data.to_csv('data_17NOV.csv', index=False)

data = pd.read_csv("/content/drive/MyDrive/NLP_learning/data_with_language.csv")
data.head()

,Lyric,Lyric_STR,No_of_Characters,Contains_letter,Lyric_detected_language
0,Kara bulutlar sardГ„В± yine dГѓВјnyamГ„В±\nKГ„...,Kara bulutlar sardГ„В± yine dГѓВјnyamГ„В±\nKГ„...,277,True,tr
1,Dolunay parlak gГѓВ¶rГѓВјnmГѓВјyor bu gece\nBe...,Dolunay parlak gГѓВ¶rГѓВјnmГѓВјyor bu gece\nBe...,471,True,tr
2,Ay Г„В±Г…ВџГ„В±Г„ВџГ„В±nГ„В±n altГ„В±nda\nYaln...,Ay Г„В±Г…ВџГ„В±Г„ВџГ„В±nГ„В±n altГ„В±nda\nYaln...,660,True,tr
3,Ay Г„В±Г…ВџГ„В±Г„ВџГ„В±nГ„В±n altГ„В±nda\nYaln...,Ay Г„В±Г…ВџГ„В±Г„ВџГ„В±nГ„В±n altГ„В±nda\nYaln...,660,True,tr
4,YГѓВјrГѓВјyorum yalnГ„В±z baГ…ВџГ„В±ma\nNereye...,YГѓВјrГѓВјyorum yalnГ„В±z baГ…ВџГ„В±ma\nNereye...,516,True,tr


In [6]:
# Removing non-english data

data = data.drop(data[data['Lyric_detected_language'] != 'en'].index)

data.head(15)

,Lyric,Lyric_STR,No_of_Characters,Contains_letter,Lyric_detected_language
7,[GGFH cover]\n\n,[GGFH cover]\n\n,14,True,en
8,[Originally performed by Bloodthorn]\n\nDrops ...,[Originally performed by Bloodthorn]\n\nDrops ...,685,True,en
9,Electro eroticism\nIntelligence is sexy\nElect...,Electro eroticism\nIntelligence is sexy\nElect...,539,True,en
10,Inhale/exhale\nInhale the exhaled\nInhale/exha...,Inhale/exhale\nInhale the exhaled\nInhale/exha...,502,True,en
11,Under the dark sky we stand\nUnder heavy rain ...,Under the dark sky we stand\nUnder heavy rain ...,838,True,en
12,"Seducer sun shine on me\nFamiliar scene, fever...","Seducer sun shine on me\nFamiliar scene, fever...",551,True,en
13,The black iris devours pictures in silence\nTh...,The black iris devours pictures in silence\nTh...,722,True,en
14,Night swells to distant spheres\nSilent throat...,Night swells to distant spheres\nSilent throat...,739,True,en
15,Opulent kinetic sculptures\nVisualized voices ...,Opulent kinetic sculptures\nVisualized voices ...,561,True,en
18,The nights fall and skies fall\nDown here we t...,The nights fall and skies fall\nDown here we t...,264,True,en


In [7]:
# Clean from /n and other stufff 
global cnt_total, cnt_kept
cnt_total, cnt_kept = 0, 0
def check_line(text):
  # cnt_total += 1
  if len(text) < 5: return False
  black_list = ['\x03', '\x05', '\t', '\n', '\x0c', '\x10', '\x19', '#', '$', '%', '&', '(', ')', '*', '+', '<', '>', '_', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '@', '[', '\\', ']', '^', '{', '|', '}', '~', '\x7f', '\xa0', '¤', '¦', '§', '©', '«', '¬', '\xad', '®', '°', '±', 'µ', '¶', '·', '»', 'в', 'г', 'д', 'к', 'о', 'п', 'ш', 'щ', 'ё', 'ђ', 'ѓ', 'є', 'ї', 'љ', 'њ', 'ћ', 'ќ', 'ў', 'џ', 'ґ', '†', '‡', '•', '‰', '‹', '›', '€', '№', '™']
  for ch in black_list:
    if ch in text: return False
  # cnt_kept += 1
  return True

def clean_line(text):
  return re.sub("[^A-Za-z0-9\ \n]","",text.lower())
  
def clean_song(text):
  lines = text.split('\n')
  lines = [clean_line(l) for l in lines if check_line(l)] 
  return ' : '.join(lines)



In [8]:
data['Lyrics'] = data['Lyric_STR'].apply(clean_song)

print(cnt_total, cnt_kept)

0 0


In [ ]:
data['Lyrics'].head(5)

The generation of text with RNN involves the following workflow.  

<p style="background-color:#B3C5E3;font-family:newtimeroman;color:#444160;text-align:center;font-size:120%;">Loading Data ➡️ Preprocessing ➡️ Building Mapping Dictionary ➡️ Building Model ➡️ Generating Text</p>

As I have loaded and explored the data,  I will proceed further by pre-processing the text.  


<a id="4"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">DATA PREPROCESSING</p>

**In this section, I will be performing the following:**

**Creating a Corpus of Lyrics text:** For the model, we need a sequence of the text string. I am creating a Corpus out of Lyrics column. 

**Removing the unrequired characters that may have sneaked in my text corpus:** The data cleaning process for NLP is crucial preprocessing. To do that, I look into the Corpus to check for what this Corpus is comprised of. That is, all the unique symbols present. After examining the Corpus, I will be eliminating any foreign language or irrelevant symbols from the Corpus. 

**Creating a dictionary to map characters and their indices:** The computer doesn’t understand the text. For the computer, the text is just a cluster of symbols. It works with numbers. So we create a dictionary to map each unique character in our Corpus to a number and vice versa. This will be used to encode and decode the information going in and getting out of the RNN

**Splitting the corpus into smaller sentences of equal length:** Encoding and splitting the corpus into smaller sequences of equal length: At this point, Corpus contain only intended characters (i.e, lower cap English alphabets, Numbers and a few punctuations). We will encode this corpus and create small sequences of equal lengths of features and the corresponding targets. Each feature and target will contain the mapped index in the dictionary of the unique characters they signify. 

The labels are then resized and normalized. Whereas the targets are one-hot encoded. Ready to be sent to the RNN for the training, but before that let us built the RNN model. 

**Creating a Corpus**

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
data_1, data_2 = train_test_split(data, test_size=0.7)



In [ ]:
# split data into 2 halves

# data_1 = data.iloc[:, :45369]

In [ ]:
print(len(data_1))

In [ ]:
#Lining up all the lyrics to create corpus

Corpus =' ; '.join([song.lower() for song in data_1.Lyrics])
    
# Corpus = Corpus.lower() #converting all alphabets to lowercase 
print("Number of unique characters:", len(set(Corpus)))

In [ ]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

In [ ]:
# data_1.Lyrics[5][:500]

In [ ]:
# data.to_csv('data_17NOV.csv', index=False)


The total number of unique characters present in the Corpus clearly shows, that some of the foreign language scripts have sneaked in. I will take a look at all the characters present. I will then remove the unrequired characters. 

**Creating a list of sorted unique characters**

In [ ]:
# # Storing all the unique characters present in my corpus to bult a mapping dic. 
# symb = sorted(list(set(Corpus)))

# L_corpus = len(Corpus) #length of corpus
# L_symb = len(symb) #length of total unique characters

# #Building dictionary to access the vocabulary from indices and vice versa
# mapping = dict((c, i) for i, c in enumerate(symb))
# reverse_mapping = dict((i, c) for i, c in enumerate(symb))

# print("Total number of characters:", L_corpus)
# print("Number of unique characters:", L_symb)

In [ ]:
# #Splitting the Corpus in equal length of strings and output target
# length = 40
# features = []
# targets = []
# for i in range(0, L_corpus//100 - length, 1):
#     feature = Corpus[i:i + length]
#     target = Corpus[i + length]
#     features.append([[mapping[j]/ float(L_symb) for j in feature]])
#     targets.append(mapping[target])
    
    
# L_datapoints = len(targets)
# print("Total number of sequences in the Corpus:", L_datapoints)

**Encoding the Labels and Targets**

In [ ]:
import numpy as np

In [ ]:
# # reshape X and normalize
# X = (np.reshape(features, (L_datapoints, length, 1)))# / float(L_symb)

# # one hot encode the output variable
# y = np_utils.to_categorical(targets)

In [ ]:
# y = np_utils.to_categorical(targets)

In [ ]:
# X.shape
# # X[0]
# y.shape

<a id="5"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;font-size:150%;color:#444160;text-align:center;border-radius:20px 60px;">MODEL BUILDING</p>


Recurrent Neural Networks are pretty popular with generating text. In this project, I will be using a LSTM Model, an improved version of a standard recurrent neural network

**Following steps are involved in the model building**

* Initialising the Model
* Defining by adding layers
* Compiling the Model
* Training the Model

**Building the Model**

In [ ]:
# #Initialising the Model

# model = Sequential()
# #Adding layers
# model.add(LSTM(256, input_shape=(length, 1), return_sequences=True))
# model.add(LSTM(256))
# model.add(Dense(y.shape[1], activation='softmax'))
# #Compiling the model for training  
# opt = Adamax(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt)

# #Model's Summary               
# model.summary()
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()


In [ ]:
# #Training the Model
# history = model.fit(X, y, batch_size=512, epochs=100, verbose=2)

In [ ]:
# from keras.models import load_model
# #To be used later; I am saving the model 
# model.save("Lyrics_Generator.h5")

<a id="6"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;font-size:150%;color:#444160;text-align:center;border-radius:20px 60px;">EVALUATING MODELS</p>

Now that I have my model trained on the songs lyrics let us see how it performs. I hope it creates some sensible song.

**To evaluate my model, I shall be having a look at:**
* The performance of the model via Learning Curves
* The outcome text it generates

**Plotting the learning curve for the loss function**

In [ ]:
# history_df = pd.DataFrame(history.history)
# #Plotting the learnings 

# fig = plt.figure(figsize=(15,4), facecolor="#B291B6")
# fig.suptitle("Learning Plot of Model for Loss")
# pl=sns.lineplot(data=history_df["loss"],color="#444160")
# pl.set(ylabel ="Training Loss")
# pl.set(xlabel ="Epochs")

**Generating the songs**

In [ ]:
# # The function to generate text from model
# def Lyrics_Generator(starter,Ch_count): #,temperature=1.0):
#     generated= ""
#     starter = starter 
#     seed=[mapping[char] for char in starter]
#     generated += starter 
#     # Generating new text of given length
#     for i in range(Ch_count):
#         seed=[mapping[char] for char in starter]
#         x_pred = np.reshape(seed, (1, len(seed), 1))
#         x_pred = x_pred/ float(L_symb)
#         prediction = model.predict(x_pred, verbose=0)[0]  
#         # Getting the index of the next most probable index
#         prediction = np.asarray(prediction).astype('float64')
#         prediction = np.log(prediction) / 1.0 
#         exp_preds = np.exp(prediction)
#         prediction = exp_preds / np.sum(exp_preds)
#         probas = np.random.multinomial(1, prediction, 1)
#         index = np.argmax(prediction)
#         next_char = reverse_mapping[index]  
#         # Generating new text
#         generated += next_char
#         starter = starter[1:] + next_char
       
    # return generated

Let us finally see the outcome by putting a seed in from one of my old blogposts [blogpost](https://karnikakapoor.blogspot.com/2017/04/killers-confession.html)

In [ ]:
# #Generating a song from the model
# song_1 = Lyrics_Generator("the shoe shrunk and the school belt got ", 400)
# #Let's have a look at the song
# # My_song(song_1)

In [ ]:
# song_1

Another song generated by a seed of the lyrics of a song that's stuck in my head today. (Sunflower by Shannon Purser)

In [ ]:
# #Generating a song from the model using a song out of the corpus
# song_2 = Lyrics_Generator("i'm a sunflower, a little funny", 400)
# #Let's have a look at the song
# My_song(song_2)

In [ ]:
# data.to_csv("data_processed.csv", index=False, header=True)

In [ ]:
# # =====================

# import tensorflow as tf

# # keras module for building LSTM 
# from keras_preprocessing.sequence import pad_sequences
# from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense, Bidirectional 
# from keras.preprocessing.text import Tokenizer
# from keras.callbacks import EarlyStopping
# from keras.models import Sequential

In [ ]:
# df = pd.read_csv("/content/data_processed.csv")



In [ ]:
# df.head(1)

In [ ]:
# import numpy as np
# import torch
# from torch import nn
# import torch.nn.functional as F
# from torch.autograd import Variable

In [ ]:
# DP_text = data['Lyric_STR'].str.cat(sep='\n').lower()

In [ ]:
print(Corpus[:100])
print('corpus length:', len(Corpus))

In [ ]:
chars = sorted(list(set(Corpus)))
print(chars)
print('total chars:', len(chars))

In [ ]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

print(char_to_int)

In [ ]:
import numpy as np

seq_length = 50 # The sentence window size
step = 1 # The steps between the windows
sentences = []
next_chars = []

# Create Target and sentences window
for i in range(0, len(Corpus) - seq_length, step):
    sentences.append(Corpus[i: i + seq_length]) # range from current index to sequence length charaters
    next_chars.append(Corpus[i + seq_length]) # the next character

sentences = np.array(sentences)
next_chars = np.array(next_chars)

#Print Sentence Window and next charaters
print('Sentence Window')
print (sentences[:5])
print('Target charaters')
print (next_chars[:5])
print('Number of sequences:', len(sentences))

In [ ]:
def getdata(sentences, next_chars):
    X = np.zeros((len(sentences),seq_length))
    y = np.zeros((len(sentences)))
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            X[i, t] = char_to_int[char]
        y[i] = char_to_int[next_chars[i]]
    return X, y

In [ ]:
train_x,train_y = getdata(sentences, next_chars)
print('Shape of training_x:', train_x.shape)
print('Shape of training_y:', train_y.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class SimpleLSTM(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, dropout=0.2, **kwargs):
        super(SimpleLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout, num_layers=2)
        self.embeddings = nn.Embedding(n_vocab, embedding_dim)
        self.fc = nn.Linear(hidden_dim, n_vocab)
        self.kwargs = kwargs

    def forward(self, seq_in):
        # for LSTM, input should be (Sequnce_length,batchsize,hidden_layer), so we need to transpose the input
        embedded = self.embeddings(seq_in.t())
        lstm_out, _ = self.lstm(embedded)
        # Only need to keep the last character
        ht=lstm_out[-1]
        out = self.fc(ht)
        return out

In [ ]:
X_train_tensor = torch.tensor(train_x, dtype=torch.long).cuda()
Y_train_tensor = torch.tensor(train_y, dtype=torch.long).cuda()

In [ ]:
from torch.utils.data import Dataset, DataLoader
train = torch.utils.data.TensorDataset(X_train_tensor,Y_train_tensor)
train_loader = torch.utils.data.DataLoader(train, batch_size = 128)

In [ ]:
model = SimpleLSTM(len(chars), 256, 256, char_to_int=char_to_int, int_to_char=int_to_char)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002) # Using Adam optimizer

In [ ]:
print(len(chars))

In [ ]:
import time # Add time counter
avg_losses_f = []
n_epochs = 20

for epoch in range(n_epochs):
    start_time = time.time()
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    avg_loss = 0.
    for i, (x_batch, y_batch) in enumerate(train_loader):
        y_pred = model(x_batch)

        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        avg_loss+= loss.item() / len(train_loader)

    elapsed_time = time.time() - start_time
    print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, elapsed_time))

    avg_losses_f.append(avg_loss)

print('All \t loss={:.4f} \t '.format(np.average(avg_losses_f)))

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
sentence = 'i read in the news that the average man please kis'

variance = 0.3
generated = ''
original = sentence
window = sentence

for i in range(400):
    x = np.zeros((1, seq_length))
    for t, char in enumerate(window):
        x[0, t] = char_to_int[char] # Change the sentence to index vector shape (1,50)

    x_in = Variable(torch.LongTensor(x).cuda())
    pred = model(x_in)
    pred = np.array(F.softmax(pred, dim=1).data[0].cpu())
    next_index = sample(pred, variance)
    next_char = int_to_char[next_index] # index to char

    generated += next_char
    window = window[1:] + next_char # Update Window for next char predict

print(original + generated)

In [ ]:
torch.save(model, f'/content/tims.model')